# MultipleNegativesRankingLoss
* sentence_transformers MultipleNegativesRankingLoss [[link]](https://sbert.net/docs/package_reference/sentence_transformer/losses.html#multiplenegativesrankingloss)
* 한 배치 내에서 postive 외 나머지 샘플을 negative로 간주하는 방법
    * great loss function if you only have positive pairs

In [2]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from datasets import Dataset

In [3]:
model_dir = "/Users/id4thomas/models/text_embedders/ModernBERT-korean-large-preview"
model = SentenceTransformer(model_dir)

You try to use a model that was created with version 3.3.1, however, your version is 3.3.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [20]:
dataset = Dataset.from_dict({
    "anchor": ["It's nice weather outside today.", "He drove to work."],
    "positive": ["It's so sunny.", "He took the car to the office."],
})
loss_fn = losses.MultipleNegativesRankingLoss(model)
loss_fn

MultipleNegativesRankingLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: ModernBertModel 
    (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  )
  (cross_entropy_loss): CrossEntropyLoss()
)

In [21]:
trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=dataset,
    loss=loss_fn,
)

In [22]:
dataloader = trainer.get_eval_dataloader(dataset)

In [ ]:
for x in dataloader:
    print(x)
    inputs = trainer._prepare_inputs(x)
    print(inputs)
    features, labels = trainer.collect_features(inputs)
    print(features, labels)
    ## MultipleNegativesRankingLoss doesn't use 'labels'
    loss = loss_fn(features, labels)
    print(loss)
    # loss, logits, labels = trainer.prediction_step(model, x, True)

{'anchor_input_ids': tensor([[50281,  1147,   434,  5322,  8588,  3345,  3063,    15, 50282],
        [50281,  1328, 12668,   281,   789,    15, 50282, 50283, 50283]],
       device='mps:0'), 'anchor_attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]], device='mps:0'), 'positive_input_ids': tensor([[50281,  1147,   434,   594, 32650,    15, 50282, 50283, 50283, 50283],
        [50281,  1328,  2335,   253,  1113,   281,   253,  3906,    15, 50282]],
       device='mps:0'), 'positive_attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='mps:0')}
{'anchor_input_ids': tensor([[50281,  1147,   434,  5322,  8588,  3345,  3063,    15, 50282],
        [50281,  1328, 12668,   281,   789,    15, 50282, 50283, 50283]],
       device='mps:0'), 'anchor_attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]], device='mps:0'), 'positive_input_ids': tensor([[50281,  1147,   434